In [1]:
'''HiCARS/HERA/MARFA netCDF 3 data viewer.  Interactively view the contents of released radar data.

    Copyright (C) 2022  Duncan A. Young

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.
'''

'''Production of this notebook was supported by NSF grant RISE-2127606 to D. Young'''


'''Requirements:
python3
netCDF4
matplotlib
numpy
pyproj

all can be obtained by 'pip3 install <module>''
'''
from netCDF4 import Dataset, MFDataset
from matplotlib import dates as mdates
import matplotlib
matplotlib.use('TkAgg')
%matplotlib
from matplotlib import pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.text import Text
from matplotlib.ticker import AutoLocator
from cftime import num2pydate, date2num
import numpy as np
import os
import glob
from matplotlib import dates as mdates
from pyproj import CRS, Transformer

def read_granule(path):
    '''Reads a UTIG formatted NetCDF-3 radar granule.'''
    data = Dataset(path,'r',format='NETCDF3_CLASSIC')
    if 'amplitude_high_gain' in data.variables:        
        return data
    else:
        return None,

def read_radargram(pst):
    '''Reads and compiles a full transect from UTIG radar granules.''' 
    P = pst.split('/')[0]
    S = pst.split('/')[1]
    T = pst.split('/')[2]
    
    radargram = None
    UTC_times = None
    
    print(f'Reading the following granules for {pst}')
    
    for path in list(sorted(glob.glob(f'IR2HI1B_*_{P}_{S}_{T}_*.nc'))):
        
        print(path)
        data = read_granule(path)
    
        if data is None:
            continue
    
        #rotates radargram to allow concatenation, as each granual is only 4000 records long
        radar = np.flipud(np.rot90(data['amplitude_high_gain']))
        time = data['time']
        UTC_Time = num2pydate(time,units=time.units,calendar=time.calendar)
        fasttime = data['fasttime']
        altitude = data['altitude'][:]
        
        #extracts along track information
        x, y, distance = get_along_track(data)
        
        if radargram is None:
            print(f'No radargram for {pst}')
            radargram = radar
            UTC_Times = UTC_Time
            altitudes = altitude
            xs = x
            ys = y
            distances = np.array(distance)
        else:
            radargram = np.append(radargram,radar, axis = 1)
            UTC_Times = np.append(UTC_Times,UTC_Time)
            altitudes = np.append(altitudes,altitude)
            xs = np.append(xs,x)
            ys = np.append(ys,y)
            last_distance = distances[-1]
            distances = np.append(distances, distance + last_distance)
            distances = distances
            
    return radargram, UTC_Times, altitudes, distances, xs, ys, data




Using matplotlib backend: MacOSX


In [2]:
def get_along_track(data,epsg=3031):
    '''Gets along track distance as well as PS71 coordinates.'''
    ll = CRS.from_epsg(4326)
    xy = CRS.from_epsg(epsg)
    lltoxy = Transformer.from_crs(4326, 3031,always_xy=True)
    
    lat = data['lat']
    lon = data['lon']
    xs,ys = lltoxy.transform(lon,lat)
    
    ds = [0]
    for i,x in enumerate(xs):
        if np.isnan(x):
            xs[i] = xs[i-1]
            ys[i] = ys[i-1]
        
        if i > 0:
            d = ds[i-1] + np.hypot((xs[i]-xs[i-1]),(ys[i]-ys[i-1]))
            if np.isnan(d):
                ds.append(ds[i-1])
            else:
                ds.append(d)
    return xs/1000, ys/1000, ds

In [3]:
def depth_correct(radargram,altitudes,top_elevation=3000):
    '''Autopicks the surface and depth corrects the radargram using positioning information.
    If a record has missing position data, we blank out the record.
    '''   
    bytrace = np.fliplr(np.rot90(radargram,k=3))
    rolled = np.zeros(np.shape(bytrace))
    srf = []
    rng = []
    correction = []
    
    for i,trace in enumerate(bytrace):

        #pick the surface, repick if noisy
        srf.append(np.argmax(trace))
        if srf[i] - srf[i-1] > 15:
            srf[i] = srf[i-1] + (np.argmax(trace[srf[i-1]-5:srf[i-1]+5]))
        
        #fix bad altitudes     
        if np.isnan(altitudes[i]):
            altitudes[i] = altitudes[i-1]

        #get the range to the surface in meters  
        rng.append(srf[i] * 3)
        
        #blank out trace above surface
        trace[0:srf[i]] = 100
        
        #blank out traces where the altitude is still bad after fixes
        if np.isnan(altitudes[i]):
            rolled[i] = 100
            correction.append(np.nan)
            continue
    
        #do the depth correction by rolling the record up or down
        rolled_trace = np.roll(trace,int(-srf[i]))
        correction.append(int((rng[i] + (top_elevation - altitudes[i]))/1.6))
        rolled[i] = np.roll(rolled_trace,correction[i])
        
        #blank out places where the radargram has rolled off the top of the record.
        if correction[i] > 0:
            rolled[i][:correction[i]] = 100
        else:
            rolled[i][3200+correction[i]:-1] = 100
    return rolled


def get_aspect_ratio(fig,ax,UTC_Times,distances):
    '''Get the aspect ratio of the plotted data, as well as the indexs of the first and last records of the plot.'''
    bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    
    x_axis_values = mdates.date2num(UTC_Times)
    xmin,xmax = ax.get_xlim()
    ymin,ymax = ax.get_ylim()
    
    xmin_idx = np.abs(x_axis_values-xmin).argmin()
    xmax_idx = np.abs(xmax - x_axis_values).argmin()
    xmin_distance = distances[xmin_idx]/1000
    xmax_distance = distances[xmax_idx]/1000
    
    elevation_range = ymax - ymin
    show_distance = xmax_distance - xmin_distance
    
    elevation_scale = (elevation_range/1000)/bbox.height
    distance_scale = (show_distance)/bbox.width
    return distance_scale/elevation_scale, xmin_idx, xmax_idx
        

def plot_radargram(pst,top_elevation=3600, x_index_min=None, x_index_max=None, y_index_min=None, y_index_max=None):
    '''Interactively plot the radargram.  The top elevation represents the top of the depth correct radargram.  
    The maximum depth is 5120 m below the top elevation.  The various index terms are not yet implemented.'''
    
    radargram, UTC_Times, altitudes, distances, eastings, northings, data  = read_radargram(pst) 
    depth_corrected = depth_correct(radargram, altitudes, top_elevation = top_elevation)

    #set up the plotting environment
    fig,ax = plt.subplots(nrows=1,ncols=1,figsize=(16,6))
    x_min = mdates.date2num(UTC_Times[0])
    x_max = mdates.date2num(UTC_Times[-1])
    
    #plot the initial radargram - to invert the radargram colortable change bone to bone_r
    #Uses time for the x-axis
    print(x_min,x_max,top_elevation-5120,top_elevation)
    ax.imshow(np.flipud(np.rot90(depth_corrected)),extent=(x_min,x_max,top_elevation-5120,top_elevation),vmin=100,vmax=180,cmap='bone',aspect='auto')
    ax.set_xlim(UTC_Times[0],UTC_Times[-1])
    ax.set_ylabel('Elevation (WGS-84 m)')
    ax.set_xlabel(f'UTC Time on {UTC_Times[1].strftime("%Y-%m-%d")}')
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
    
    #obtain information about the intital aspect ratio and plot the title
    aspect_ratio, xmin_idx, xmax_idx = get_aspect_ratio(fig,ax,UTC_Times,distances)
    ax.set_title(f'{pst} (Aspect ratio = {aspect_ratio:.1f})')
    
    #plot a PS71 context plot in the upper left corner showing the full flight path
    context = fig.add_axes([0.7,0.7,0.2,0.2], anchor='NE', zorder=1, aspect='equal')
    context.plot(eastings, northings,color='lightgray')
    
    #print out a text caption based on attributes in the last granule of the transect.
    print('-------------CAPTION---------------------')
    print(f"Transect {pst} was collected using the {data.instrument} radar on flight {data.flight} which took place on {UTC_Times[1].strftime('%d %B, %Y')}. {data.funding}. The investigators were {data.investigators}. {data.processing}")
    print('-----------------------------------------')
    

    def zoom(event):  
        '''Responds to a zoom or pan event, and updates the plot.'''
        
        print('')
        print(f'New view of {pst}')
        
        #deletes existing information in context plot
        for artist in context.get_children():
            if isinstance(artist, Line2D) or isinstance(artist, Text):
                try:
                    artist.remove()
                except NotImplementedError:
                    pass
        
        #deletes existing x axis information
        ax.set_xticks([])
        ax.set_xticks([],minor=True)
        
        #obtains updated aspect ratio information and updates title
        aspect_ratio, xmin_idx, xmax_idx = get_aspect_ratio(fig,ax,UTC_Times,distances)
        ax.set_title(f'{pst} (Aspect ratio = {aspect_ratio:.1f})')
        
        #prints out record information required to remake this plot
        print('Bounding radar records and elevation ranges for this plot:')
        print(xmin_idx, xmax_idx, ax.get_ylim()[0], ax.get_ylim()[1])
        
        #update the context plot making A-A' as the ends of the plotted line (in orange), superimposted on the full transect
        context.plot(eastings, northings,color='lightgray')
        context.plot(eastings[xmin_idx:xmax_idx], northings[xmin_idx:xmax_idx],color='orange')
        context.text(eastings[xmin_idx], northings[xmin_idx],'A', fontweight='bold', color='orange')
        context.text(eastings[xmax_idx], northings[xmax_idx],'A\'',fontweight='bold', color='orange')
        context.text(0, 0,'A', fontweight='bold', fontsize='large', color='orange', bbox=dict(facecolor='white'), horizontalalignment='left',verticalalignment='bottom', transform=ax.transAxes, )
        context.text(1, 0,'A\'', fontweight='bold', fontsize='large', color='orange', bbox=dict(facecolor='white'), horizontalalignment='right',verticalalignment='bottom', transform=ax.transAxes)
        
        #Calculate the appropriate x-axis to show; below 100 km length, it shows 10 km labeled ticks; below 10 km it shows 1 km labeled ticks; otherwise the plot shows time
        plot_distance = distances[xmax_idx]-distances[xmin_idx]
        print(f'Distance Shown: {plot_distance/1000:.1f} km')
        x_axis_values = mdates.date2num(UTC_Times)
        if plot_distance < 100000:
            print('Changing X Axis from time to distance')
            x_ticks = []
            x_ticks_minor = []
            x_labels = []
            x_labels_minor = []
            for tick in range(0,1*round(plot_distance/1000),1):
                index = np.argmin(np.abs(((distances - distances[xmin_idx])/1000) - tick))
                x_ticks_minor.append(x_axis_values[index])
                x_labels_minor.append(tick)
                if tick % 10 == 0:
                    x_ticks.append(x_axis_values[index])
                    x_labels.append(tick)
            if plot_distance > 10000:
                ax.set_xticks(x_ticks_minor,minor=True)
                ax.set_xticks(x_ticks)
                ax.set_xticklabels(x_labels)
            else:
                ax.set_xticks(x_ticks_minor)
                ax.set_xticklabels(x_labels_minor)
            ax.set_xlabel('Distance along track (km)')
        else:
            print('Changing X Axis from distance to time')
            ax.set_xticks([],minor=True)
            ax.set_xlabel('')
            ax.xaxis.set_major_locator(AutoLocator())
            ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
            ax.set_xlabel(f'UTC Time on {UTC_Times[0].strftime("%Y-%m-%d")}')
    
    #callbacks to connect to the GUI
    ax.callbacks.connect('xlim_changed', zoom)
    ax.callbacks.connect('ylim_changed', zoom)

In [4]:
plot_radargram('CXA1/MKB2n/F18T02a')

Reading the following granules for CXA1/MKB2n/F18T02a
IR2HI1B_2023026_CXA1_MKB2n_F18T02a_000.nc
No radargram for CXA1/MKB2n/F18T02a
IR2HI1B_2023026_CXA1_MKB2n_F18T02a_001.nc
IR2HI1B_2023026_CXA1_MKB2n_F18T02a_002.nc
19383.881678512014 19383.91500714338 -1520 3600
-------------CAPTION---------------------
Transect CXA1/MKB2n/F18T02a was collected using the High Capability Airborne Radar Sounder (HiCARS 2) radar on flight CXA1/F18 which took place on 26 January, 2023. This line was funded as part of COLDEX  NSF's Center for Oldest Ice Exploration (COLDEX) Science and Technology Center. The investigators were COLDEX  D. A. Young, D. D. Blankenship, M. Kerr, S. Singh (Universtity of Texas). PROCESSING PARAMETERS
Mode name: pik1
Goal: emulate incoherent radar acquisition parameters while rejecting clutter, quick processing with no dependencies
Filtering: 10 MHz narrow band notch filter to eliminate LO leakage
Range compression: Frequency domain convolution of overscaled synthetic chirp wave

In [5]:
#plot_radargram('CLX2/MKB2n/X90a')

In [6]:
#plot_radargram('CXA1/MKB2n/F19T02a')


New view of CXA1/MKB2n/F18T02a
Bounding radar records and elevation ranges for this plot:
6760 11210 -1520.0 3600.0
Distance Shown: 80.1 km
Changing X Axis from time to distance

New view of CXA1/MKB2n/F18T02a
Bounding radar records and elevation ranges for this plot:
6760 11210 -1342.6839826839823 3123.4632034632036
Distance Shown: 80.1 km
Changing X Axis from time to distance
